https://www.kaggle.com/code/tuckerarrants/text-generation-with-huggingface-gpt2

https://huggingface.co/docs/transformers/model_doc/gpt2

https://huggingface.co/openai-community/gpt2-medium


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/TFM-MUECIM/*.py /content
!cp /content/drive/MyDrive/TFM-MUECIM/*.txt /content
#!cp /content/drive/MyDrive/TFM-MUECIM/*.dat /content
#!cp /content/drive/MyDrive/TFM-MUECIM/*.tar /content
!cp /content/drive/MyDrive/TFM-MUECIM/50LabelsGPT2DataFrameForTextGen.csv /content
#!cd /content; tar xf data.tar data
!cd /content/drive/MyDrive/TFM-MUECIM


In [3]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [4]:
import os
import shutil
import torch
from datetime import datetime
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel, set_seed
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

https://jalammar.github.io/illustrated-gpt2/


In [5]:
def tokenize_function(trainDataFrame):
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
  tokenizer.pad_token = tokenizer.eos_token
  return tokenizer(trainDataFrame['text'], padding="max_length", truncation=True)

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

def train(train_file_path,
          model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          last_epoch_trained,
          save_steps):

  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset('csv', data_files=train_file_path)
  train_dataset = train_dataset.map(tokenize_function, batched=True)
  train_dataset = train_dataset['train'].select_columns(['input_ids'])

  data_collator = load_data_collator(tokenizer)
  tokenizer.save_pretrained(output_dir)
  tokenizer.pad_token = tokenizer.eos_token

  if last_epoch_trained > 0:
    model = GPT2LMHeadModel.from_pretrained(output_dir)
  else:
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  # Move the model to the correct device before training.
  model.to(device)

  training_args = TrainingArguments(
          output_dir=output_dir,
          report_to = 'none',
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=1,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset
  )

  trainer.train()
  trainer.save_model(output_dir='GPT2_trained_model_202504')

  baseDir = '.'
  prefixDate = datetime.today().strftime('%Y%m%d')
  fileName = f'{prefixDate}_50L_tfm_GPT2_model_epoch_{last_epoch_trained + 1}.pt'
  modelFullPath = os.path.join(baseDir,fileName)
  drivePath = '/content/drive/MyDrive/TFM-MUECIM'
  destFullPath = os.path.join(drivePath,fileName)
  print('Save model after epoch train session')
  torch.save(model, modelFullPath)
  shutil.copyfile(modelFullPath, destFullPath)

  model.save_pretrained(save_directory='GPT2_trained_model_202504')

  return model

In [ ]:
dataFrame = '50LabelsGPT2DataFrameForTextGen.csv'
train_file_path = f'/content/{dataFrame}'
model_name = 'gpt2'
output_dir = '/content/result'
overwrite_output_dir = False

per_device_train_batch_size = 4
num_train_epochs = 1 # 5.0
save_steps = 500

torch.cuda.empty_cache()

model = train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    last_epoch_trained=0,
    save_steps=save_steps
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,1.093000
1000,0.902900
1500,0.840800
2000,0.797800
2500,0.729600
3000,0.741000


In [ ]:
!tar czf GPT2_trained_model_202504.tar.gz GPT2_trained_model_202504
!cp GPT2_trained_model_202504.tar.gz /content/drive/MyDrive/TFM-MUECIM